# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [2]:
import findspark
findspark.init("/home/stuthi/prep/spark-2.0.0-bin-hadoop2.7/")
from pyspark.sql import SparkSession
sp = SparkSession.builder.appName('Hyundai').getOrCreate()

In [6]:
df = sp.read.csv('cruise_ship_info.csv',header=True, inferSchema=True)

In [8]:
df.describe().show()

+-------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
|summary|               Age|           Tonnage|       passengers|           length|            cabins|passenger_density|             crew|
+-------+------------------+------------------+-----------------+-----------------+------------------+-----------------+-----------------+
|  count|               158|               158|              158|              158|               158|              158|              158|
|   mean|15.689873417721518| 71.28467088607599|18.45740506329114|8.130632911392404| 8.830000000000005|39.90094936708861|7.794177215189873|
| stddev| 7.615691058751413|37.229540025907866|9.677094775143416|1.793473548054825|4.4714172221480615| 8.63921711391542|3.503486564627034|
|    min|                 4|             2.329|             0.66|             2.79|              0.33|             17.7|             0.59|
|    max|                48

In [11]:
df.head(2)

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55),
 Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55)]

In [12]:
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [13]:
from pyspark.ml.regression import LinearRegression

In [14]:
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler

In [15]:
df.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [17]:
assm = VectorAssembler(inputCols=['Age','Tonnage','passengers','length','cabins','passenger_density']
                       ,outputCol='features')

In [18]:
feature_df = assm.transform(df)

In [19]:
feature_df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)
 |-- features: vector (nullable = true)



In [20]:
feature_df.select('features').show(2)

+--------------------+
|            features|
+--------------------+
|[6.0,30.276999999...|
|[6.0,30.276999999...|
+--------------------+
only showing top 2 rows



In [21]:
feature_label_data = feature_df.select(['features','crew'])

In [25]:
train_data, test_data = feature_label_data.randomSplit([0.75,0.25])

In [26]:
train_data.describe().show()


+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               119|
|   mean|7.9994117647058935|
| stddev|3.2325904745993714|
|    min|              0.59|
|    max|              21.0|
+-------+------------------+



In [27]:
test_data.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|               39|
|   mean|7.167948717948716|
| stddev|4.210635768651607|
|    min|             0.59|
|    max|             13.6|
+-------+-----------------+



In [28]:
lr = LinearRegression(labelCol='crew')

In [29]:
lrmodel = lr.fit(train_data)

In [30]:
lrmodel.coefficients

DenseVector([-0.0092, 0.0213, -0.1785, 0.3606, 0.812, -0.0055])

In [31]:
lrmodel.intercept

-0.11790475527117561

In [34]:
test_results = lrmodel.evaluate(test_data)

In [35]:
test_results.rootMeanSquaredError

0.6451050682125492

In [36]:
test_results.r2

0.9759094582022918

In [37]:
pred_labels = lrmodel.transform(test_data.select('features'))

In [39]:
pred_labels.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[5.0,115.0,35.74,...|   11.419006366708|
|[6.0,112.0,38.0,9...|10.881579995171368|
|[6.0,113.0,37.82,...|11.395628459574795|
|[6.0,158.0,43.7,1...|13.871998201420435|
|[7.0,116.0,31.0,9...| 12.62579954564438|
|[7.0,158.0,43.7,1...|13.815939467439875|
|[9.0,59.058,17.0,...| 7.487147601765868|
|[9.0,81.0,21.44,9...| 9.518450084200843|
|[9.0,113.0,26.74,...| 11.49041861383856|
|[9.0,116.0,26.0,9...|11.372950557474354|
|[11.0,90.0,22.4,9...|10.055893274355238|
|[11.0,91.62700000...| 9.447163172998557|
|[11.0,138.0,31.14...| 13.24394340391883|
|[12.0,2.329,0.94,...|0.9497822563585263|
|[12.0,25.0,3.88,5...| 2.984927112804535|
|[12.0,58.6,15.66,...| 7.350057186867383|
|[12.0,88.5,21.24,...| 9.686573942513135|
|[13.0,30.27699999...| 3.862508150837122|
|[13.0,63.0,14.4,7...| 6.943507973141674|
|[13.0,85.619,21.1...| 9.618629600706939|
+--------------------+------------

In [40]:
# Now using the categorical variables as well.. 

In [42]:
from pyspark.ml.feature import StringIndexer

In [43]:
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [44]:
si = StringIndexer(inputCol='Cruise_line',outputCol='cruiseIndex')
indexed_df = si.fit(df).transform(df)

In [45]:
indexed_df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)
 |-- cruiseIndex: double (nullable = true)



In [46]:
from pyspark.ml.feature import VectorAssembler

In [48]:
assembler = VectorAssembler(inputCols=['Age','Tonnage','passengers','length',
                                       'cabins','passenger_density','cruiseIndex'],
                           outputCol = 'features')

In [50]:
feature_df = assembler.transform(indexed_df)

In [51]:
feature_df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)
 |-- cruiseIndex: double (nullable = true)
 |-- features: vector (nullable = true)



In [52]:
train_dt, test_dt = feature_df.randomSplit([0.75, 0.25])

In [55]:
train_dt.describe().show()

+-------+------------------+-----------------+-----------------+------------------+------------------+-----------------+------------------+-----------------+
|summary|               Age|          Tonnage|       passengers|            length|            cabins|passenger_density|              crew|      cruiseIndex|
+-------+------------------+-----------------+-----------------+------------------+------------------+-----------------+------------------+-----------------+
|  count|               124|              124|              124|               124|               124|              124|               124|              124|
|   mean|15.991935483870968|69.88117741935486|18.07032258064517| 8.076048387096769| 8.686370967741942|40.34814516129032| 7.661935483870972|5.064516129032258|
| stddev| 7.379928798367538| 35.9230006987729|9.517763447013547|1.6882052906358018|4.3663108527554595|8.882370807715063|3.3632796318448896| 4.76051157919742|
|    min|                 4|            3.341|      

In [56]:
test_dt.describe().show()

+-------+------------------+-----------------+------------------+-----------------+-----------------+-----------------+------------------+------------------+
|summary|               Age|          Tonnage|        passengers|           length|           cabins|passenger_density|              crew|       cruiseIndex|
+-------+------------------+-----------------+------------------+-----------------+-----------------+-----------------+------------------+------------------+
|  count|                34|               34|                34|               34|               34|               34|                34|                34|
|   mean|14.588235294117647|76.40329411764706|19.869117647058825| 8.32970588235294|9.353823529411764|            38.27| 8.276470588235295|5.0588235294117645|
| stddev| 8.446328401033819|41.83161770684322| 10.25996437103125|2.151296276306264|4.868417087474266|7.584582847441048|3.9911780613360954|4.8238119998957965|
|    min|                 5|            2.329|      

In [57]:
lr = LinearRegression(labelCol='crew')

In [59]:
lrmodel2 = lr.fit(train_dt.select('features','crew'))

In [61]:
test_res = lrmodel2.evaluate(test_dt.select('features','crew'))

In [62]:
test_res.rootMeanSquaredError

1.405192387414615

In [63]:
test_res.r2

0.8722872235603272

In [64]:
unlabelled_preds = lrmodel2.transform(test_dt.select('features'))

In [65]:
unlabelled_preds.show(10)

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[12.0,138.0,31.14...|12.940015146334389|
|[12.0,58.6,15.66,...| 7.414801635179585|
|[19.0,16.8,2.96,5...|2.2646274493984864|
|[11.0,110.0,29.74...|12.062189577618398|
|[11.0,91.0,20.32,...| 9.265425504706222|
|[7.0,116.0,31.0,9...|12.601715570825409|
|[11.0,90.0,22.4,9...|10.071368874748462|
|[23.0,48.563,20.2...| 5.649151892457253|
|[27.0,12.5,3.94,4...|0.9848827739596251|
|[17.0,77.71300000...| 8.483445167177916|
+--------------------+------------------+
only showing top 10 rows



In [76]:
print(lrmodel2.coefficients)

[-0.0114326645165,0.00792594559234,-0.149914360323,0.472166985545,0.857145352855,0.00290908365324,0.0518938958641]


In [67]:
from pyspark.sql.functions import corr

In [69]:
df.select(corr('crew','passengers')).show()

+----------------------+
|corr(crew, passengers)|
+----------------------+
|    0.9152341306065384|
+----------------------+



In [71]:
df.select(corr('crew','cabins')).show()

+------------------+
|corr(crew, cabins)|
+------------------+
|0.9508226063578497|
+------------------+



In [72]:
df.select(corr('crew','length')).show()

+------------------+
|corr(crew, length)|
+------------------+
|0.8958566271016579|
+------------------+



In [74]:
indexed_df.select(corr('crew','cruiseIndex')).show()

+-----------------------+
|corr(crew, cruiseIndex)|
+-----------------------+
|   -0.48332562728617057|
+-----------------------+

